In [5]:
#Imports
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image
import easygui
import pytesseract

In [6]:
#1 Open a user-selected image
f = easygui.fileopenbox(filetypes=["*.jpg","*.jpeg","*.png"])#open user file select
I = cv2.imread(f)#set a file to work with
Original = I.copy()#create a copy incase theres a problem

In [7]:
Img = cv2.imread("Cards.jpg")

In [8]:

#Path to your Tesseract-OCR
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract'
#Imput takes small section of top corner of card
image = cv2.imread('sampleLetter.png',0)
#Gets a threshold
thresh = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)[1]
#Blurs it with threshold
imBlur = cv2.GaussianBlur(thresh, (5,5), 0)
imBlur = 255 - imBlur

cv2.imshow("imBlur", imBlur)
key = cv2.waitKey(0)
#convert to text from section of image
textFromImage = pytesseract.image_to_string(imBlur, lang='eng',config='--psm 6')
print("The card is: ",textFromImage)

The card is:  K

